In [2]:
#import libraries for math and message-passing interface
import numpy as np
import mpi4py
from mpi4py import MPI



ModuleNotFoundError: No module named 'mpi4py'

In [3]:
comm = MPI.COMM_WORLD

# Rank of each processor
rank = comm.Get_rank()

#Number of processors
size = comm.Get_size()

#Function to find the Max and Min prime numbers
def PrimeNumber_calc(buf):
    Prime = []
    for number in buf:  
        if number > 1:  
            for i in range (2, int(number)):  
                if (number % i) == 0:  
                    break  
            else:  
                Prime.append(number)

    max_min = [min(Prime), max(Prime)]
    
    # Returns a list of two integers (<min>, <max>)
    return max_min


#setup integer array of N=1,000,000 digits
N = np.arange(0,10000,1)

# Get total size of array
array_size = len(N)

#Divide array_size into P equal chunks
chunk = int(array_size/size)

#Initialize Temp Variables for intermediate data
part_n = np.zeros(chunk)
worker_chunk = np.zeros(chunk)
part_sum = np.zeros(chunk)

#Intialize Temp Variables to store local results
worker_buffer = []
temp_buffer = []
master_buffer = []

#intialize counter for iterations
i=1

if rank==0:
    #Master (rank=0) part
    #divide total work into P equal chunks 

    while i < size:
        
        #compute start and stop indices for array slice for each rank in the group
        start=i*chunk
        end = (i+1)*chunk

        #prepare work_chunk
        part_sum = N[start:end]
        
        #send work_chunk to each worker (rank>0) in the group
        comm.send(part_sum, dest=i, tag=20)

        #Receive results from each worker
        temp_buffer += comm.recv(source=i, tag=10)
        i = i+1
else:
    #Worker (rank >0) part
    #recv work from master
    worker_chunk = comm.recv(source=0, tag=20)

    # Find Max, Min Prime numbers of worker_chunk
    worker_buffer = PrimeNumber_calc(worker_chunk)

    #send result to Master
    comm.send(worker_buffer, dest=0, tag=10)

#Master aggregates all results
if rank == 0:
    master_buffer = PrimeNumber_calc(N[0:chunk]) 
    total = master_buffer + temp_buffer
    Min_Max = PrimeNumber_calc(total)
    print ("[<MIN>,<MAX>] = ", Min_Max)


NameError: name 'MPI' is not defined